<a href="https://colab.research.google.com/github/pranav172/Invoice-Fraud-Detection-Computer-Vision-/blob/main/InvoiceFraudDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

osamahosamabdellatif_high_quality_invoice_images_for_ocr_path = kagglehub.dataset_download('osamahosamabdellatif/high-quality-invoice-images-for-ocr')
pranav1718_invoicefrauddetection_path = kagglehub.notebook_output_download('pranav1718/invoicefrauddetection')

print('Data source import complete.')


In [ ]:
# =======================================================
# STEP 1: RESTORE SAVED MODELS
# =======================================================
import shutil
import os

# --- PASTE YOUR CORRECT PATH HERE ---
# Based on your screenshot, the models are in the root of your notebook's output directory.
# Find the directory named after your notebook in the "Input" panel and copy its path.
INPUT_MODEL_PATH = "/kaggle/input/invoicefrauddetection/" # <-- THIS IS THE MOST LIKELY PATH
OUTPUT_MODEL_PATH = "/kaggle/working/"

print(f"Attempting to copy models from: {INPUT_MODEL_PATH}")

try:
    # Create the destination directory if it doesn't exist
    os.makedirs(OUTPUT_MODEL_PATH, exist_ok=True)

    # Copy the model files
    shutil.copy(os.path.join(INPUT_MODEL_PATH, "MobileNetV2_best.h5"), OUTPUT_MODEL_PATH)
    shutil.copy(os.path.join(INPUT_MODEL_PATH, "DenseNet121_best.h5"), OUTPUT_MODEL_PATH)

    print("Models restored successfully!")
    print("Files in current working directory:", os.listdir(OUTPUT_MODEL_PATH))
except FileNotFoundError:
    print("\nERROR: Could not find the model files.")
    print("If this fails, please find the directory named 'invoicefrauddetection' (or similar) in your 'Input' panel, right-click it, copy the path, and paste it into the INPUT_MODEL_PATH variable.")
except Exception as e:
    print(f"\nAn unexpected error occurred: {e}")



In [ ]:
# BLOCK 1: SETUP & DATA ORGANIZATION (PATH CORRECTED)
# ==============================================================================
print("Block 1: Setting up the environment and organizing data...")

# --- Stage 1.1: Import Libraries ---
import os
import shutil
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont, ImageFilter

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0, DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

print("Libraries imported successfully.")

In [ ]:
# --- Stage 1.2: Define Paths ---
# Path has been corrected based on your screenshot.
# We are starting with the images from the 'batch1_1' folder.
BASE_DATA_PATH = "/kaggle/input/high-quality-invoice-images-for-ocr/batch_1/batch_1/batch1_1" # <-- PATH CORRECTED
WORKING_DIR = "/kaggle/working/"
PROCESSED_DATA_PATH = os.path.join(WORKING_DIR, "processed_data")

print(f"Attempting to read from base data path: {BASE_DATA_PATH}")
print(f"Working directory for output: {WORKING_DIR}")

In [ ]:
# --- Stage 1.3: Create Directories for our new Dataset ---
NOT_FRAUD_DIR = os.path.join(PROCESSED_DATA_PATH, "not_fraud")
FRAUD_DIR = os.path.join(PROCESSED_DATA_PATH, "fraud")

if os.path.exists(PROCESSED_DATA_PATH):
    shutil.rmtree(PROCESSED_DATA_PATH)
os.makedirs(NOT_FRAUD_DIR, exist_ok=True)
os.makedirs(FRAUD_DIR, exist_ok=True)
print(f"Created directories for processed data at: {PROCESSED_DATA_PATH}")

In [ ]:
# --- Stage 1.4: Copy Original Images and Validate ---
try:
    if os.path.exists(BASE_DATA_PATH) and os.path.isdir(BASE_DATA_PATH):
        original_image_files = [f for f in os.listdir(BASE_DATA_PATH) if f.endswith('.jpg')]
        for file_name in original_image_files:
            shutil.copy(os.path.join(BASE_DATA_PATH, file_name), NOT_FRAUD_DIR)

        num_copied_files = len(os.listdir(NOT_FRAUD_DIR))
        print(f"Successfully copied {num_copied_files} original images to the 'not_fraud' directory.")

        # --- Stage 1.5: Validate and Explore Sample ---
        if num_copied_files == 0:
            print("\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print("!!! ERROR: The source directory was found, but it contains no .jpg files.  !!!")
            print(f"!!! Please check the contents of: {BASE_DATA_PATH} !!!")
            print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        else:
            print("Displaying a sample legitimate invoice...")
            sample_image_path = os.path.join(NOT_FRAUD_DIR, random.choice(os.listdir(NOT_FRAUD_DIR)))
            sample_image = Image.open(sample_image_path)
            plt.figure(figsize=(8, 10))
            plt.imshow(sample_image)
            plt.title("Sample Legitimate Invoice")
            plt.axis('off')
            plt.show()
    else:
        print("\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print("!!! ERROR: The specified BASE_DATA_PATH does not exist or is not a directory. !!!")
        print(f"!!! Please check the path: {BASE_DATA_PATH} !!!")
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

except Exception as e:
    print(f"\nAn unexpected error occurred: {e}")


print("\n--- Block 1 Execution Finished ---")

In [ ]:
# ==============================================================================
# BLOCK 2: SYNTHETIC FRAUD GENERATION (REVISED)
# ==============================================================================
#
# INDUSTRY INSIGHT (REVISED):
# Our first attempt at data generation was too subtle. The model failed to learn.
# We will now create much more obvious visual anomalies. This gives the model a
# stronger, clearer signal to learn from, which is often necessary when a
# model struggles to converge. We are making the difference between "fraud" and
# "not_fraud" unmistakable.
#
print("\nBlock 2 (Revised): Generating new, more obvious fraudulent data...")

# --- Stage 2.1: Define NEW Fraud Generation Functions ---

def add_black_bars(image_path, output_path):
    """
    Simulates redacting information with a thick black marker. This is a
    very strong and obvious visual artifact for the model to detect.
    """
    with Image.open(image_path) as img:
        img = img.convert("RGB")
        draw = ImageDraw.Draw(img)
        img_w, img_h = img.size

        # Draw 1 to 3 black bars in random positions
        for _ in range(random.randint(1, 3)):
            # Choose a random position and size
            bar_x = random.randint(int(img_w * 0.1), int(img_w * 0.8))
            bar_y = random.randint(int(img_h * 0.1), int(img_h * 0.8))
            bar_w = int(img_w * random.uniform(0.2, 0.5))
            bar_h = int(img_h * random.uniform(0.03, 0.06))
            draw.rectangle([bar_x, bar_y, bar_x + bar_w, bar_y + bar_h], fill="black")

        img.save(output_path, "JPEG")

def add_void_stamp(image_path, output_path):
    """
    Adds a large, semi-transparent "VOID" or "FAKE" stamp across the image.
    This is another unmistakable sign of an altered document.
    """
    with Image.open(image_path) as img:
        img = img.convert("RGB")

        # Create a transparent layer for the stamp
        stamp_layer = Image.new("RGBA", img.size, (255, 255, 255, 0))
        draw = ImageDraw.Draw(stamp_layer)
        img_w, img_h = img.size

        # Load a large font
        try:
            font = ImageFont.truetype("LiberationSans-Regular.ttf", size=int(img_w / 6))
        except IOError:
            font = ImageFont.load_default()

        # Choose stamp text and color
        stamp_text = random.choice(["VOID", "FAKE", "COPY"])
        text_color = (255, 0, 0, 100) # Red with alpha for transparency

        # Calculate text size and position
        text_bbox = draw.textbbox((0, 0), stamp_text, font=font)
        text_w = text_bbox[2] - text_bbox[0]
        text_h = text_bbox[3] - text_bbox[1]
        pos_x = (img_w - text_w) / 2
        pos_y = (img_h - text_h) / 2

        # Draw the rotated text
        draw.text((pos_x, pos_y), stamp_text, font=font, fill=text_color)

        # Rotate the stamp layer
        rotated_stamp = stamp_layer.rotate(30, expand=0)

        # Composite the stamp onto the original image
        img.paste(rotated_stamp, (0, 0), rotated_stamp)
        img.save(output_path, "JPEG")

In [ ]:
# --- Stage 2.2: Generate the NEW Fraudulent Images ---
# First, we must clean the old fraud images
if os.path.exists(FRAUD_DIR):
    shutil.rmtree(FRAUD_DIR)
os.makedirs(FRAUD_DIR, exist_ok=True)
print(f"Cleaned old fraud data and recreated directory: {FRAUD_DIR}")

source_images = os.listdir(NOT_FRAUD_DIR)
num_fraud_images_to_generate = len(source_images)

print(f"Starting generation of {num_fraud_images_to_generate} new fraudulent images...")

for i in range(num_fraud_images_to_generate):
    source_image_name = random.choice(source_images)
    source_image_path = os.path.join(NOT_FRAUD_DIR, source_image_name)

    output_image_name = f"fraud_{i}_{source_image_name}"
    output_image_path = os.path.join(FRAUD_DIR, output_image_name)

    # Use our new, more obvious techniques
    technique = random.choice([add_black_bars, add_void_stamp])
    try:
        technique(source_image_path, output_image_path)
    except Exception as e:
        print(f"Could not generate image {output_image_name} due to error: {e}")

print(f"Successfully generated {len(os.listdir(FRAUD_DIR))} new synthetic fraudulent images.")


In [ ]:
# --- Stage 2.3: Explore a Sample of the NEW Fraudulent Invoices ---
print("Displaying a sample of the new fraudulent invoices...")

plt.figure(figsize=(16, 8))
try:
    # Show one of each type
    sample_bar_path = None
    sample_stamp_path = None

    fraud_files = os.listdir(FRAUD_DIR)
    random.shuffle(fraud_files)

    for fname in fraud_files:
        if "fraud" in fname and sample_bar_path is None:
             # Just pick a random one, as we don't store the type in the name
             # We'll rely on the visual difference
             sample_bar_path = os.path.join(FRAUD_DIR, fname)
        if "fraud" in fname and sample_stamp_path is None and fname != os.path.basename(sample_bar_path):
             sample_stamp_path = os.path.join(FRAUD_DIR, fname)
        if sample_bar_path and sample_stamp_path:
            break

    plt.subplot(1, 2, 1)
    plt.imshow(Image.open(sample_bar_path))
    plt.title("Sample: Black Bar Redaction")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(Image.open(sample_stamp_path))
    plt.title("Sample: VOID Stamp")
    plt.axis('off')

    plt.show()

except Exception as e:
    print(f"Could not display sample fraud images. Error: {e}")

print("\n--- Block 2 (Revised) Execution Finished ---")
print("\nIMPORTANT: Now you must re-run Block 3, Block 4, and Block 5 to train the models on this new data.")


In [ ]:
print("\nBlock 3: Setting up the model training framework...")

# --- Stage 3.1: Define Hyperparameters ---
# These settings are crucial for model performance. We choose values that
# are known to work well for transfer learning tasks.
IMG_SIZE = (224, 224)  # The standard input size for many pre-trained models.
BATCH_SIZE = 32        # Number of images to process at a time.
EPOCHS = 8             # We'll keep this low for fast training. EarlyStopping will quit sooner if needed.
LEARNING_RATE = 0.001  # A common and effective learning rate for the Adam optimizer.

In [ ]:
# --- Stage 3.2: Create Data Generators ---
# ImageDataGenerator is a powerful Keras utility to load and augment image data.
# Augmentation (like rotation, zoom) creates more diverse training examples,
# making the final model more robust to variations in real-world data.
datagen = ImageDataGenerator(
    rescale=1./255.,          # Normalize pixel values from [0, 255] to [0, 1]
    validation_split=0.2,     # Reserve 20% of the data for validation
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    horizontal_flip=False,    # Flipping an invoice horizontally doesn't make sense
    fill_mode='nearest'
)

# Generator for the training data (uses 80% of images)
train_generator = datagen.flow_from_directory(
    PROCESSED_DATA_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',      # It's a two-class problem: fraud vs. not_fraud
    subset='training',
    shuffle=True
)

# Generator for the validation data (uses 20% of images)
validation_generator = datagen.flow_from_directory(
    PROCESSED_DATA_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

print(f"Found {train_generator.n} images for training.")
print(f"Found {validation_generator.n} images for validation.")
print(f"Classes: {train_generator.class_indices}")

In [ ]:
# --- Stage 3.3: Define a Reusable Model Creation and Training Function ---
def create_and_train_model(base_model_class, model_name, train_gen, val_gen):
    """
    Builds, compiles, and trains a model using a given base architecture.

    Args:
        base_model_class: The Keras pre-trained model class (e.g., MobileNetV2).
        model_name (str): A name for the model, used for saving files.
        train_gen: The data generator for training.
        val_gen: The data generator for validation.

    Returns:
        A tuple of (trained model, training history).
    """
    print(f"\n--- Training Model: {model_name} ---")

    # 1. Load the base model with pre-trained ImageNet weights, without the top layer.
    base_model = base_model_class(input_shape=IMG_SIZE + (3,),
                                  include_top=False,
                                  weights='imagenet')
    # 2. Freeze the base model's layers. We don't want to change its learned features yet.
    base_model.trainable = False

    # 3. Add our custom classification head. This is the part we will train.
    inputs = Input(shape=IMG_SIZE + (3,))
    x = base_model(inputs, training=False) # Run in inference mode.
    x = GlobalAveragePooling2D()(x)        # Flattens the features.
    x = Dropout(0.3)(x)                    # Regularization to prevent overfitting.
    x = Dense(128, activation='relu')(x)   # A hidden layer.
    outputs = Dense(1, activation='sigmoid')(x) # Output layer for binary classification.
    model = Model(inputs, outputs)

    # 4. Compile the model.
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    print(f"Model architecture for {model_name} created.")
    model.summary()

    # 5. Set up callbacks for robust training.
    # ModelCheckpoint saves the best version of the model during training.
    checkpoint_path = os.path.join(WORKING_DIR, f"{model_name}_best.h5")
    checkpoint = ModelCheckpoint(checkpoint_path,
                                 save_best_only=True,
                                 monitor='val_accuracy',
                                 mode='max',
                                 verbose=1)
    # EarlyStopping stops training if performance doesn't improve.
    early_stopping = EarlyStopping(monitor='val_loss',
                                   patience=3, # Stop after 3 epochs of no improvement.
                                   restore_best_weights=True)

    # 6. Train the model.
    history = model.fit(
        train_gen,
        epochs=EPOCHS,
        validation_data=val_gen,
        callbacks=[checkpoint, early_stopping]
    )

    print(f"--- Finished training {model_name} ---")
    return model, history

In [ ]:
# --- Stage 3.4: Define a Plotting Function ---
def plot_history(history, model_name):
    """Plots the training and validation accuracy and loss."""
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title(f'{model_name} - Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title(f'{model_name} - Training and Validation Loss')
    plt.show()


print("\n--- Block 3 Execution Finished ---")


In [ ]:
# ==============================================================================
# BLOCK 4: MODEL TRAINING EXECUTION
# ==============================================================================
#
# INDUSTRY INSIGHT:
# This is where the main experiment happens. We will now use the functions
# from Block 3 to train each of our candidate models sequentially. We'll store
# the results of each training run so we can compare them at the end.
#
print("\nBlock 4: Executing the training for all candidate models...")

# --- Stage 4.1: Train the Models ---
# We will create a dictionary to hold the training history of each model.
model_histories = {}

# --- Train MobileNetV2 ---
mobilenet_model, mobilenet_history = create_and_train_model(
    base_model_class=MobileNetV2,
    model_name="MobileNetV2",
    train_gen=train_generator,
    val_gen=validation_generator
)
model_histories['MobileNetV2'] = mobilenet_history
plot_history(mobilenet_history, "MobileNetV2")


# --- Train EfficientNetB0 ---
efficientnet_model, efficientnet_history = create_and_train_model(
    base_model_class=EfficientNetB0,
    model_name="EfficientNetB0",
    train_gen=train_generator,
    val_gen=validation_generator
)
model_histories['EfficientNetB0'] = efficientnet_history
plot_history(efficientnet_history, "EfficientNetB0")


# --- Train DenseNet121 ---
densenet_model, densenet_history = create_and_train_model(
    base_model_class=DenseNet121,
    model_name="DenseNet121",
    train_gen=train_generator,
    val_gen=validation_generator
)
model_histories['DenseNet121'] = densenet_history
plot_history(densenet_history, "DenseNet121")


print("\n--- All models have been trained. ---")
print("The best version of each model has been saved to a .h5 file in the /kaggle/working/ directory.")
print("\n--- Block 4 Execution Finished ---")


In [ ]:
#BLOCK 5: FINE-TUNING THE BEST PERFORMING MODEL
# ==============================================================================
#
# INDUSTRY INSIGHT:
# The initial training (Block 4) gave poor results because the pre-trained
# base model is frozen. Its features are general-purpose. Now, we perform
# fine-tuning. We unfreeze the top layers of the base model and retrain with a
# very low learning rate. This allows the model to adapt its specialized
# feature detectors to our specific dataset (invoices) without corrupting
# its valuable pre-trained knowledge. This is a critical step for achieving
# high accuracy in transfer learning.
#
print("\nBlock 5: Fine-tuning the best model (MobileNetV2)...")

# --- Stage 5.1: Load the Best Saved Model ---
# We start from the best weights we saved during the initial training phase.
best_model_path = os.path.join(WORKING_DIR, "MobileNetV2_best.h5")
model_to_fine_tune = tf.keras.models.load_model(best_model_path)

print("Successfully loaded the best MobileNetV2 model for fine-tuning.")

# --- Stage 5.2: Unfreeze Layers of the Base Model ---
# We need to access the original base model within our loaded model.
# It's usually the second layer (index 1), after the Input layer.
base_model = model_to_fine_tune.layers[1]
base_model.trainable = True

# We'll unfreeze the top layers and keep the bottom ones frozen.
# The bottom layers detect very simple features (edges, colors), which are
# universal. The top layers detect more complex features (shapes, textures)
# which we want to adapt.
fine_tune_at = 100 # Unfreeze from the 100th layer onwards.

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

print(f"Unfroze layers from layer {fine_tune_at} onwards in the base model.")

# --- Stage 5.3: Re-compile the Model with a Low Learning Rate ---
# This is CRITICAL. Using a low learning rate prevents the model from
# destroying the pre-trained weights with large updates.
LOW_LEARNING_RATE = 0.00001

model_to_fine_tune.compile(
    optimizer=Adam(learning_rate=LOW_LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_to_fine_tune.summary()

# --- Stage 5.4: Continue Training (Fine-Tuning) ---
# We'll train for a few more epochs to let the model adapt.
fine_tune_epochs = 10
total_epochs = EPOCHS + fine_tune_epochs

print("\nStarting fine-tuning...")

history_fine_tune = model_to_fine_tune.fit(
    train_generator,
    epochs=total_epochs,
    initial_epoch=mobilenet_history.epoch[-1] + 1, # Start from where we left off
    validation_data=validation_generator,
    callbacks=[ # We can reuse the same callbacks
        ModelCheckpoint(os.path.join(WORKING_DIR, "MobileNetV2_finetuned_best.h5"),
                        save_best_only=True, monitor='val_accuracy', mode='max'),
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    ]
)

print("--- Finished fine-tuning ---")

# --- Stage 5.5: Visualize the Fine-Tuning Results ---
# We can append the fine-tuning history to the original history for a complete plot.
mobilenet_history.history['accuracy'].extend(history_fine_tune.history['accuracy'])
mobilenet_history.history['val_accuracy'].extend(history_fine_tune.history['val_accuracy'])
mobilenet_history.history['loss'].extend(history_fine_tune.history['loss'])
mobilenet_history.history['val_loss'].extend(history_fine_tune.history['val_loss'])

plot_history(mobilenet_history, "MobileNetV2 (with Fine-Tuning)")


print("\n--- Block 5 Execution Finished ---")

In [ ]:
# =======================================================
# STEP 1: RESTORE SAVED MODELS
# =======================================================
import shutil
import os

# --- DEBUGGING: List available input directories ---
# This will help you find the correct folder name for your saved models.
print("Searching for your saved model directory...")
print("Available directories in /kaggle/input/:")
try:
    for item in os.listdir("/kaggle/input/"):
        print(f"  - {item}")
except FileNotFoundError:
    print("Could not find the /kaggle/input/ directory.")

print("-" * 50)


# --- PASTE YOUR CORRECT PATH HERE ---
# Look at the list above. Find the directory that looks like your notebook's name.
# Copy that name and append it to "/kaggle/input/".
# For example, if you see '- invoicefrauddetection', your path should be '/kaggle/input/invoicefrauddetection/'
INPUT_MODEL_PATH = "/kaggle/input/invoicefrauddetection/" # <-- REPLACE THIS VALUE
OUTPUT_MODEL_PATH = "/kaggle/working/"

print(f"Attempting to copy models from: {INPUT_MODEL_PATH}")

try:
    # Create the destination directory if it doesn't exist
    os.makedirs(OUTPUT_MODEL_PATH, exist_ok=True)

    # Copy the model files
    shutil.copy(os.path.join(INPUT_MODEL_PATH, "MobileNetV2_best.h5"), OUTPUT_MODEL_PATH)
    shutil.copy(os.path.join(INPUT_MODEL_PATH, "DenseNet121_best.h5"), OUTPUT_MODEL_PATH)

    print("\nModels restored successfully!")
    print("Files in current working directory:", os.listdir(OUTPUT_MODEL_PATH))
except FileNotFoundError:
    print("\nERROR: Could not find the model files.")
    print("Please look at the list of 'Available directories' printed above.")
    print("Update the INPUT_MODEL_PATH variable with the correct path and re-run this cell.")
except Exception as e:
    print(f"\nAn unexpected error occurred: {e}")



In [ ]:
# ==============================================================================
# FINAL BLOCK: LAUNCH THE WEB APPLICATION WITH A PUBLIC URL
# ==============================================================================
#
# INDUSTRY INSIGHT:
# When running a web server inside a cloud environment like Kaggle or Colab,
# the default URLs (like localhost) are not accessible from the public internet.
# We need to use a tunneling service to create a secure public URL that forwards
# traffic to our application. `ngrok` is the industry standard for this.
#
print("Final Block: Installing libraries and launching the app with a public tunnel...")

# --- Step 1: Install necessary libraries ---
# We use -q to make the output less noisy (quiet mode)
!pip install -q streamlit pyngrok

# --- Step 2: Write the app.py file (if not already done) ---
# This ensures the app file exists before we try to run it.
app_code = """
import streamlit as st
from PIL import Image
import numpy as np
import tensorflow as tf
import os

st.set_page_config(page_title="Invoice Fraud Detector", page_icon="🕵️", layout="centered")

@st.cache_resource
def load_keras_model(model_path):
    try:
        return tf.keras.models.load_model(model_path)
    except Exception as e:
        st.error(f"Error loading model {os.path.basename(model_path)}: {e}")
        return None

st.title("Invoice Fraud Detection 🕵️")
st.write('''
This application uses deep learning to determine if an uploaded invoice image
shows signs of visual tampering (fraud). Choose a model and upload an image to get a prediction.
''')

MODEL_DIR = "/kaggle/working/"
MODEL_FILES = {
    "MobileNetV2": os.path.join(MODEL_DIR, "MobileNetV2_best.h5"),
    "DenseNet121": os.path.join(MODEL_DIR, "DenseNet121_best.h5")
}

available_models = [name for name, path in MODEL_FILES.items() if os.path.exists(path)]

if not available_models:
    st.error("Could not find any trained model files. Please ensure the training blocks have been run successfully.")
else:
    model_name = st.selectbox("Choose a detection model:", available_models)
    uploaded_file = st.file_uploader("Upload an invoice image...", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None:
        image = Image.open(uploaded_file).convert("RGB")

        # Display the uploaded image (FIXED: use_container_width)
        st.image(image, caption="Uploaded Invoice", use_container_width=True)

        image_resized = image.resize((224, 224))
        image_array = np.array(image_resized) / 255.0
        image_array = np.expand_dims(image_array, axis=0)

        with st.spinner(f"Loading {model_name} and analyzing image..."):
            model = load_keras_model(MODEL_FILES[model_name])
            if model:
                prediction = model.predict(image_array)
                score = prediction[0][0]
                st.subheader("Analysis Complete!")
                if score > 0.5:
                    st.success(f"**Result:** This invoice appears to be **LEGITIMATE**.")
                    st.metric(label=f"Confidence (Legitimate)", value=f"{score:.2%}")
                else:
                    st.error(f"**Result:** This invoice shows signs of **FRAUD!**")
                    st.metric(label=f"Confidence (Fraudulent)", value=f"{1 - score:.2%}")
                with st.expander("See model output details"):
                    st.write(f"Model: `{model_name}` | Raw score: `{score:.4f}`")
"""
with open("app.py", "w") as f:
    f.write(app_code)

# --- Step 3: Authenticate and Run ngrok ---
import subprocess
import time
from pyngrok import ngrok

# !! IMPORTANT !!
# 1. Go to https://dashboard.ngrok.com/get-started/your-authtoken
# 2. Copy your authtoken.
# 3. Paste it below, replacing "YOUR_AUTHTOKEN_HERE".
NGROK_AUTH_TOKEN = "304BU2pMYLdmOKmW4bS0mBNrKxJ_B4PrzzQXQNQdbCm35SyD"

if NGROK_AUTH_TOKEN == "YOUR_AUTHTOKEN_HERE":
    print("ERROR: Please paste your ngrok authtoken into the NGROK_AUTH_TOKEN variable.")
else:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    ngrok.kill() # Terminate any existing tunnels

    print("Starting Streamlit server in the background...")
    proc = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"])

    print("Waiting for server to start...")
    time.sleep(10)

    print("Creating public tunnel with ngrok...")
    try:
        public_url = ngrok.connect(8501)
        print("\n\n====================================================================")
        print("CONGRATULATIONS! Your app is live.")
        print("Click the link below to open it in a new tab:")
        print(public_url)
        print("====================================================================")
    except Exception as e:
        print(f"An error occurred with ngrok: {e}")
        print("Please ensure your authtoken is correct.")

